#  Panóptico de Twitch v0.1.0


Aplicación de *chat-scrapping* o *minería de texto* basada en API.

## Preparación del entorno de desarrollo


En esta sección se va a inicializar la **máquina** en los **servidores de Google** Colab para realizar nuestra **monitorización**.

In [ ]:
import socket
import time
import re
import datetime
import pytz
import pandas as pd
from os import mkdir 
import os.path
from os import path

## Variables globales del proyecto

En esta función definimos las **variables globales** de nuestro proyecto. Se recomienda modificar los datos en los cuadros de *UI* a la derecha del código.

Se recomienda un **tamaño pequeño** (entre 5.000 y 10.000 mensajes) para el tamaño máximo de los archivos .csv para **minimizar** las posibilidades de **pérdida de la información**.

Se presentan dos opciones para el guardado de los datos: **local** (en la máquina virtual de Google Colab); y en **Google Drive** (recomendado).

Para obtener el token **OAuth** para acceder a la información de la API sigue las siguientes instrucciones:
1. **Creamos** una cuenta en *Twitch.tv*: https://www.twitch.tv/signup

2. **Registramos** nuestra cuenta como desarrollador: https://dev.twitch.tv/console/apps/create y establecemos *'OAuth Redirect URL'*: 'http://localhost'

3. **Adquirimos** el *OAuth* en:
https://twitchapps.com/tmi/ 

In [ ]:
tz = pytz.timezone('Europe/Paris')

#@markdown Servidor IRC al que queremos conectarnos.
server = 'irc.chat.twitch.tv' #@param {type:"string"}
port = 6667

#@markdown Nombre de la cuenta con la que queremos conectarnos.
nickname = "repurposing_twitch" #@param {type:"string"}

#@markdown Token OAuth de la cuenta.
token = "oauth:" #@param {type:"string"}

#@markdown Canal que queremos monitorizar (¡en minúsculas!).
channel = "lvpes" #@param {type:"string"}

#@markdown Duración de la monitorización en segundos.
duration = 60 #@param {type:"integer"}

#@markdown Tamaño máximo de los CSV
tam_csv = 5000 #@param {type:"integer"}

#@markdown ¿Guardar datos en el Drive personal?
save_to_gdrive = True #@param {type:"boolean"}

#@markdown Modo debug
debug_mode = True #@param {type:"boolean"}

if save_to_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')
  output_dir = "drive/MyDrive/Panoptico_Twitch/" + channel + "/" + datetime.datetime.now().strftime("%Y_%m_%d") + "/"

  if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

    
else:
    output_dir = "/content/" + channel + "/"
    mkdir(output_dir)

Mounted at /content/drive


## Funciones auxiliares

Las siguientes funciones están dedicadas a la **segmentación** del texto enviado por la API a través de **expresiones regulares** (regex).

In [ ]:
def parse_msg(msg, debug=True):
    try:
        pat_message = re.compile(
            fr"@badge-info=(?P<badge_info>[^;]*).*badges=(?P<badges>[^;]*).*first-msg=(?P<first_msg>[^;]*).*mod=(?P<mod>[^;]*).*subscriber=(?P<subscriber>[^;]*).+:(?P<username>[\d\w]+)![^:]+:(?P<text>.*)", 
            flags=re.IGNORECASE
        )

        msg_data = pat_message.search(msg).groupdict()
        
        msg_data["months_as_sub"] = 0
        msg_data["medal"] = 0
        msg_data["bits"] = 0
        msg_data["sub-gifter"] = 0
        msg_data["mentions"] = ""
        msg_data["predictions"] = ""
        msg_data["amazon_prime"] = 0
        msg_data["vip"] = 0

        badge_info_list = msg_data["badge_info"].split(",")
        for badge_info in badge_info_list:
          if "subscriber" in badge_info:
            msg_data["months_as_sub"] = int(badge_info.split('/')[-1])
          if "predictions" in badge_info:
            msg_data["predictions"] = badge_info.split('/')[-1]

        badge_list = msg_data["badges"].split(",")
        interest_list = ["subscriber", "bits", "sub-gifter", "premium"]
        for badge in badge_list:
          if "subscriber" in badge:
            msg_data["medal"] = int(badge.split('/')[-1])
          if "bits" in badge:
            msg_data["bits"] = int(badge.split('/')[-1])
          if "sub-gifter" in badge:
            msg_data["sub-gifter"] = int(badge.split('/')[-1])
          if "premium" in badge:
            msg_data["amazon_prime"] = int(badge.split('/')[-1])
          if "vip" in badge:
            msg_data["vip"] = int(badge.split('/')[-1])

        text_list = msg_data["text"].split(" ")
        for text_info in text_list:
          if "@" in text_info:
            msg_data["mentions"] = str(text_info.split('/')[-1])            

#@badge-info=predictions/VISITANTE,subscriber/4;badges=predictions/blue-3,subscriber/3,moments/2;color=#1E90FF;display-name=DidacticDani;emotes=;first-msg=0;flags=;id=bb3eb7c0-6322-4297-8370-97d35a4b20c9;mod=0;returning-chatter=0;room-id=48878319;subscriber=1;tmi-sent-ts=1669839182999;turbo=0;user-id=255326102;user-type= :didacticdani!didacticdani@didacticdani.tmi.twitch.tv PRIVMSG #thegrefg :lo peor es que son 2

        date = datetime.datetime.now(tz).strftime("%m/%d/%Y")
        time = datetime.datetime.now(tz).strftime("%H:%M:%S")
        msg_data["date"] = str(date)
        msg_data["time"] = str(time)

        if debug:
            print(msg_data)
        return msg_data
    except Exception as e:
        return None

## Función principal de monitorización


La siguiente función es nuestro **main**. Realiza, en primer lugar, la comunicación mediante envío de sockets a la API enviando nuestras credenciales y entra en el bucle descifrando los mensajes recibidos y tabulándolos a través de las funciones auxiliares hasta que finalice el tiempo indicado.

In [ ]:
server_socket = socket.socket()
server_socket.connect((server, port))

if (debug_mode):
  print(server_socket)

server_socket.send(f"PASS {token}\n".encode('utf-8'))
server_socket.send(f"NICK {nickname}\n".encode('utf-8'))
server_socket.send(f"JOIN #{channel}\n".encode('utf-8'))
server_socket.send(f"CAP REQ :twitch.tv/tags\n".encode('utf-8'))

response = server_socket.recv(16384).decode('utf-8')

if (debug_mode):
  print(response)

response = server_socket.recv(16384).decode('utf-8')

if (debug_mode):
  print(response)

msg_list = []
start_time = time.time()
fragment_counter = 0
bucle_time = time.time()

while (time.time() - start_time) < duration:
  try:
    response = server_socket.recv(1024).decode('utf-8')
    if response.startswith("PING"):
      server_socket.send(f"PONG\n".encode('utf-8'))
    else:
      for m in response.split("\r\n"):
        msg_data = parse_msg(m, debug=debug_mode)
        if msg_data is not None:
          msg_list.append(msg_data)

          if (debug_mode):
            print(len(msg_list), "/", tam_csv)

          if len(msg_list) >= tam_csv:

            if debug_mode:
              print(f"Volcando el fragmento {fragment_counter}")
            
            df_msg = pd.DataFrame(msg_list)
            df_msg.to_csv(output_dir + channel + datetime.datetime.now().strftime("_%Y_%m_%d_%Hh%M") + "_" + str(fragment_counter) + '.csv', index=False)
            fragment_counter += 1
            msg_list = []    

          if (time.time() - bucle_time) > 3600: 
            if debug_mode:
              print(f"Ha pasado una hora. Volcando el fragmento {fragment_counter}")
            
            df_msg = pd.DataFrame(msg_list)
            df_msg.to_csv(output_dir + channel + datetime.datetime.now().strftime("_%Y_%m_%d_%Hh%M") + "_" + str(fragment_counter) + '.csv', index=False)
            fragment_counter += 1
            msg_list = []
            bucle_time = time.time()

  except Exception as e:
    print("Ha ocurrido una excepción, guardamos la información")
    df_msg = pd.DataFrame(msg_list)
    df_msg.to_csv(output_dir + channel + datetime.datetime.now().strftime("_%Y_%m_%d_%Hh%M") + "_" + str(fragment_counter) + '.csv', index=False)

    if debug_mode:
      print(f"Volcando el fragmento {fragment_counter}")
      
    fragment_counter += 1
    msg_list =[]
    bucle_time = time.time()

server_socket.close()

df_msg = pd.DataFrame(msg_list)
df_msg.to_csv(output_dir + channel + datetime.datetime.now().strftime("_%Y_%m_%d_%Hh%M") + "_" + str(fragment_counter) + '.csv', index=False)

<socket.socket fd=46, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('172.28.0.12', 38276), raddr=('44.226.36.141', 6667)>
:tmi.twitch.tv 001 repurposing_twitch :Welcome, GLHF!
:tmi.twitch.tv 002 repurposing_twitch :Your host is tmi.twitch.tv
:tmi.twitch.tv 003 repurposing_twitch :This server is rather new
:tmi.twitch.tv 004 repurposing_twitch :-
:tmi.twitch.tv 375 repurposing_twitch :-
:tmi.twitch.tv 372 repurposing_twitch :You are in a maze of twisty passages, all alike.
:tmi.twitch.tv 376 repurposing_twitch :>

:repurposing_twitch!repurposing_twitch@repurposing_twitch.tmi.twitch.tv JOIN #lvpes
:tmi.twitch.tv CAP * ACK :twitch.tv/tags
:repurposing_twitch.tmi.twitch.tv 353 repurposing_twitch = #lvpes :repurposing_twitch
:repurposing_twitch.tmi.twitch.tv 366 repurposing_twitch #lvpes :End of /NAMES list

{'badge_info': 'subscriber/5', 'badges': 'subscriber/3', 'first_msg': '0', 'mod': '0', 'subscriber': '1', 'username': 'zumodebrocoli', 'text': '@agrishad li

Para la limpieza y unificación de la base de datos: https://drive.google.com/file/d/1tXV2XKBkFMQDIJT6BHqn6OvT6DtA1Nmv/view?usp=sharing
